In [1]:
pip install --upgrade datasets

In [2]:
import pandas as pd
from datasets import Dataset
from huggingface_hub.utils import get_session
import string
from nltk.corpus import stopwords


In [3]:
from datasets import load_dataset

dataset = load_dataset("imdb")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
df = load_dataset('imdb')

In [5]:
df

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [6]:
df['train'].features

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['neg', 'pos'], id=None)}

In [7]:
df_train = pd.DataFrame(df['train'])
df_test = pd.DataFrame(df['test'])

In [8]:
from transformers import AutoTokenizer

#tokenizing text data
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize_function(examples):
    # padding and truncation control for variable length sequences
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# apply to dataset
tokenized_datasets = df.map(tokenize_function, batched=True)

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [9]:
#Creating dataset samples for the model
sample_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(100))
sample_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(100))

In [10]:
#Loading distilBERT for fine tuning
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
 !apt-get install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


In [34]:
#Creating huggingface repo
from huggingface_hub import notebook_login
notebook_login()

In [13]:
# pip install transformers[torch]

In [14]:
# pip install accelerate -U

In [27]:
from transformers import TrainingArguments, Trainer
import torch

repo_name = "nbreaz/text-sentiment-analysis-imdb"

training_args = TrainingArguments(
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    output_dir=repo_name,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=sample_train_dataset,
    eval_dataset=sample_train_dataset,
    tokenizer=tokenizer
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss


TrainOutput(global_step=7, training_loss=0.5350429330553327, metrics={'train_runtime': 4.8868, 'train_samples_per_second': 20.463, 'train_steps_per_second': 1.432, 'total_flos': 13246739865600.0, 'train_loss': 0.5350429330553327, 'epoch': 1.0})

In [28]:
trainer.evaluate()

{'eval_loss': 0.509570300579071,
 'eval_runtime': 1.7461,
 'eval_samples_per_second': 57.271,
 'eval_steps_per_second': 4.009,
 'epoch': 1.0}